---
**Observation:**  

1. Null values are present in the following columns:
   - **`videoDefaultLanguage`**  (will be dropped after data cleaning)
   - **`videoDefaultAudioLanguage`** 
   - **`channelCountry`**

---

2. The following columns will be dropped as part of data cleaning:
   - **`videoDescription`**: Reserved for analysis in future NLP project with a broader dataset.  
   - **`videoLiveBroadcastContent`**: All values are `'none'`, providing no variability or insights. 
   - **`videoFavoriteCount`**: All values are `0`, making it redundant.  
   - **`videoTags`**: Reserved for analysis in future NLP project with a broader dataset.  
   - **`videoUniqueId`**: Identified as a duplicate column.  
   - **`channelIdUnique`**: Identified as a duplicate column.  
   - **`channelTitleCheck`**: Identified as a duplicate column.  
   - **`channelDescription`**: Reserved for analysis in future NLP project with a broader dataset.
---

3. The columns **`channelName`** and **`videoTitle`** require further processing due to the presence of:
    - Multilingual text.  
    - Emojis and special characters.  

---

In [1]:
def Source_File_Extraction(repo_url,kaggle_repo_url):
    if os.path.exists(kaggle_repo_url):
        print("Already cloned and the repo file exists")
        repo = git.Repo(kaggle_repo_url)  # Access the existing repository
        origin = repo.remote(name='origin')  # Get the remote repository
        origin.pull()  # Pull the latest changes from the repository
        print("Successfully pulled the git repo before push")
    else:
        # Clone the repository if it doesn't exist
        repo = git.Repo.clone_from(repo_url, kaggle_repo_url)
        print("Successfully cloned the git repo")
    output_files = os.listdir(source_path)
    Source_File = max([i for i in output_files if i.startswith("S_") and i.endswith('records.json')])
    Source_File = pd.read_json(f'{source_path}/{Source_File}')
    return Source_File

In [2]:
def Requirement_File_Extraction(repo_url,kaggle_repo_url):
    if os.path.exists(kaggle_repo_url):
        print("Already cloned and the repo file exists")
        repo = git.Repo(kaggle_repo_url)  # Access the existing repository
        origin = repo.remote(name='origin')  # Get the remote repository
        origin.pull()  # Pull the latest changes from the repository
        print("Successfully pulled the git repo before push")
    else:
        # Clone the repository if it doesn't exist
        repo = git.Repo.clone_from(repo_url, kaggle_repo_url)
        print("Successfully cloned the git repo")
    output_files = os.listdir(requirement_path)
    Requirement_File = max([i for i in output_files if i.startswith("RE_") and i.endswith('country_details.json')])
    Requirement_File = pd.read_json(f'{requirement_path}/{Requirement_File}')
    return Requirement_File

In [3]:
def DataCleaning(Target_File):
    # Dropped the columns
    Target_File = Target_File.drop(['videoDescription','videoLiveBroadcastContent','videoFavoriteCount','videoTags','videoUniqueId','channelIdUnique','channelTitleCheck','channelDescription'],axis=1)
    duplicates = Target_File[Target_File.duplicated(keep=False)]  # This will select all duplicates, including the first occurrence
    
    # display(duplicates)
    # print(Target_File.duplicated(subset=['videoId', 'channelId']).sum())  # Check for duplicates based on videoId and channelId
    
    # Removing Duplicates
    Target_File  = Target_File.drop_duplicates(ignore_index=True)
    
    # Remving the videos which has videoDefaultAudioLanguage as None or starts without en
    Target_File_EN = Target_File[Target_File['videoDefaultAudioLanguage'].str.startswith("en",na=False)].reset_index(drop=True)

    for i in range(len(Target_File_EN['channelName'])):
        try:
            # Check and translate non-ASCII characters
            if not Target_File_EN['channelName'][i].isascii():
                Target_File_EN.loc[i, 'channelName'] = GoogleTranslator(source='auto', target='en').translate(Target_File_EN['channelName'][i])
            if not Target_File_EN['videoTitle'][i].isascii():
                Target_File_EN.loc[i, 'videoTitle'] = GoogleTranslator(source='auto', target='en').translate(Target_File_EN['videoTitle'][i])
    
            # Remove emojis
            Target_File_EN.loc[i, 'channelName'] = emoji.replace_emoji(Target_File_EN['channelName'][i], replace='')
            Target_File_EN.loc[i, 'videoTitle'] = emoji.replace_emoji(Target_File_EN['videoTitle'][i], replace='')
    
            # Decode HTML entities like &amp; and &#39;
            Target_File_EN.loc[i, 'channelName'] = html.unescape(Target_File_EN['channelName'][i])
            Target_File_EN.loc[i, 'videoTitle'] = html.unescape(Target_File_EN['videoTitle'][i])
    
            # Remove non-ASCII characters
            Target_File_EN.loc[i, 'channelName'] = re.sub(r'[^\x00-\x7F]+', '', Target_File_EN['channelName'][i])
            Target_File_EN.loc[i, 'videoTitle'] = re.sub(r'[^\x00-\x7F]+', '', Target_File_EN['videoTitle'][i])
    
            # print(Target_File_EN['channelName'][i])
            # print(Target_File_EN['videoTitle'][i])
        except Exception as e:
            print(e)
    
    # Removing Duplicates
    Target_File_EN  = Target_File_EN.drop_duplicates(ignore_index=True)
    Target_File_EN = Target_File_EN.drop(['videoDefaultLanguage'],axis=1)
    Target_File_EN['channelCountry'] = Target_File_EN['channelCountry'].fillna('Unknown')
    return Target_File_EN


In [4]:
def FeatureEngineering(Cleaned_File):
    # Feature Engineering  
    
    # videoPublishedWeekDay - Derive the day of the week from the videoPublishedOn timestamp.  
    
    # videoDurationClassification - Categorize videos into duration segments based on videoDurationInSeconds.  
        # Categories:
        #     Very Short (0 - 60 sec) → Typically Shorts, Reels, or quick snippets.
        #     Short (61 sec - 2 min) → Brief content, short tutorials, or quick explanations.
        #     Medium (2 min 1 sec - 5 min) → Standard short-form content, concise videos.
        #     Long (5 min 1 sec - 10 min) → In-depth discussions, detailed tutorials.
        #     Very Long (10 min 1 sec - 1 hour) → Educational content, podcasts, detailed explainers.
        #     Extended (1 hour 1 sec - 3 hours) → Webinars, live sessions, long-form discussions.
        #     Ultra Long (3 hours 1 sec and above) → Movie-length content, streams, recorded conferences.
    
    # channelGrowth - Assess channel growth using factors such as channelPublishOnInSeconds, channelViewCount, channelSubscriberCount, and channelVideoCount.  
    
    # videoEngagement - Evaluate video engagement based on videoPublishedOnInSeconds, videoViewCount, videoLikeCount, and videoCommentCount.  
    
    # channelGrowthRank - Rank channels based on the channelGrowth metric.  
    
    # videoEngagementRank - Rank videos based on the videoEngagement metric.  
    
    # Geographic Classification - Assign an upper-level geographical classification.  
        # Columns include:  
            # - country_name  
            # - continent  
            # - continent_code  
            # - it_hub_country (indicator for whether the country is a major IT hub).  
       
    Country_Details_ISO = Requirement_File_Extraction(repo_url,kaggle_repo_url)
    display(Country_Details_ISO)
    return Country_Details_ISO

In [5]:
def GitHubPush(Target_File_EN):
    record_count = len(Target_File_EN)
    
    # Generate a timestamp for the file name using the current time in IST (Indian Standard Time).
    timestamp = datetime.datetime.now(ist).strftime("%Y-%m-%d_%H:%M:%S")
    
    # Create a filename using the generated timestamp to ensure uniqueness with number of records.
    filename = f"DC_{timestamp}_{record_count}_records.json"
    
    # Save the DataFrame to a JSON file with readable formatting.
    Target_File_EN.to_json(filename, orient="records", indent=4)
    print(f"DataFrame saved as {filename}")
    
    if not os.path.exists(destination_path):
        # Create the destination directory if it doesn't exist
        os.makedirs(destination_path)
        print('created the destination directory, DataCleaning/Daily')
        shutil.copyfile(f'/kaggle/working/{filename}', f'{destination_path}/{filename}')
    else:
        print('Destination directory already exists')
        shutil.copyfile(f'/kaggle/working/{filename}', f'{destination_path}/{filename}')
    
    # Initialize the repository for git operations
    repo = Repo(kaggle_repo_url)
    
    # Add the copied file to the staging area
    repo.index.add([f"{destination_path}/{filename}"])
    
    # Create a timestamp for the commit message
    timestamp = datetime.datetime.now(ist).strftime("%Y-%m-%d_%H:%M:%S")
    # Commit the changes with a message including the timestamp
    repo.index.commit(f"{timestamp} Added files from Kaggle notebook, {filename}")
    
    # Push the changes to the remote repository
    origin = repo.remote(name="origin")
    push_result = origin.push()
    if push_result:
        print("Push successful.")
    else:
        print("Push failed.")

In [6]:
def main():
    Source_File = Source_File_Extraction(repo_url,kaggle_repo_url)
    Cleaned_File = DataCleaning(Source_File)
    # display(Cleaned_File.sort_values(by='videoDurationInSeconds',ascending = True))
    Feature_File = FeatureEngineering(Cleaned_File)
    # GitHubPush(Cleaned_File)

In [7]:
if __name__ == "__main__":
    import os  
    import git 
    from git import Repo  
    import datetime  
    from pytz import timezone
    import pandas as pd
    import deep_translator
    from deep_translator import GoogleTranslator
    from pytz import timezone 
    import shutil
    import emoji
    import re
    import html
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    secret_value_0 = user_secrets.get_secret("dataCleanRepoUrl")
    repo_url = secret_value_0
    ist = timezone('Asia/Kolkata')
    kaggle_repo_url = '/kaggle/working/YouTubeFoodChannelAnalysis'
    destination_path = '/kaggle/working/YouTubeFoodChannelAnalysis/DataCleaning/Daily'
    source_path = '/kaggle/working/YouTubeFoodChannelAnalysis/Source/Daily'
    requirement_path = '/kaggle/working/YouTubeFoodChannelAnalysis/Requirement/Daily'
    # Below script prevents all columns and rows from getting truncated while display
    pd.set_option("display.max_columns", None)
    pd.set_option("display.max_rows",None)
    main()


Successfully cloned the git repo
Already cloned and the repo file exists
Successfully pulled the git repo before push


,GS,GD,CH,SL,HU,TW,WF,BB,PN,CI,TN,IT,BJ,ID,CV,KN,LA,BQ,UG,AD,BI,ZA,FR,LY,MX,GA,MP,MK,CN,YE,BL,GG,SB,SJ,FO,UZ,EG,SN,LK,PS,BD,PE,SG,TR,AF,AW,CK,GB,ZM,FI,NE,CX,TK,GW,AZ,RE,DJ,KP,MU,MS,VI,CO,GR,HR,MA,DZ,AQ,NL,SD,FJ,LI,NP,PR,GE,PK,MC,BW,LB,PG,YT,DO,NF,BV,QA,MG,IN,SY,ME,SZ,PY,SV,UA,IM,NA,AE,BG,GL,DE,KH,IQ,TF,SE,CU,KG,RU,MY,ST,CY,CA,MW,SA,BA,ET,ES,SI,OM,PM,MO,SM,LS,MH,SX,IS,LU,AR,TC,NR,CC,EH,DM,CR,AU,TH,HT,TV,HN,GQ,LC,PF,BY,LV,PW,GP,PH,GI,DK,CM,GN,BH,SR,CD,SO,CZ,NC,VU,SH,TG,VG,KE,NU,HM,RW,EE,RO,TT,GY,TL,VN,UY,VA,HK,AT,AG,TM,MZ,PA,FM,IE,CW,GF,NO,AX,CF,BF,ER,TZ,KR,JO,MR,LT,UM,SK,AO,KZ,MD,ML,FK,AM,WS,JE,JP,BO,CL,US,VC,BM,SC,IO,GT,EC,MQ,TJ,MT,GM,NG,BS,XK,KW,MV,SS,IR,AL,BR,RS,BZ,MM,BT,VE,LR,JM,PL,KY,BN,KM,GU,TO,KI,GH,TD,ZW,MF,MN,PT,AS,CG,BE,IL,NZ,NI,AI
country_name,South Georgia,Grenada,Switzerland,Sierra Leone,Hungary,Taiwan,Wallis and Futuna,Barbados,Pitcairn Islands,Ivory Coast,Tunisia,Italy,Benin,Indonesia,Cape Verde,Saint Kitts and Nevis,Laos,Caribbean Netherlands,Uganda,Andorra,Burundi,South Africa,France,Libya,Mexico,Gabon,Northern Mariana Islands,North Macedonia,China,Yemen,Saint Barthélemy,Guernsey,Solomon Islands,Svalbard and Jan Mayen,Faroe Islands,Uzbekistan,Egypt,Senegal,Sri Lanka,Palestine,Bangladesh,Peru,Singapore,Turkey,Afghanistan,Aruba,Cook Islands,United Kingdom,Zambia,Finland,Niger,Christmas Island,Tokelau,Guinea-Bissau,Azerbaijan,Réunion,Djibouti,North Korea,Mauritius,Montserrat,United States Virgin Islands,Colombia,Greece,Croatia,Morocco,Algeria,Antarctica,Netherlands,Sudan,Fiji,Liechtenstein,Nepal,Puerto Rico,Georgia,Pakistan,Monaco,Botswana,Lebanon,Papua New Guinea,Mayotte,Dominican Republic,Norfolk Island,Bouvet Island,Qatar,Madagascar,India,Syria,Montenegro,Eswatini,Paraguay,El Salvador,Ukraine,Isle of Man,Namibia,United Arab Emirates,Bulgaria,Greenland,Germany,Cambodia,Iraq,French Southern and Antarctic Lands,Sweden,Cuba,Kyrgyzstan,Russia,Malaysia,São Tomé and Príncipe,Cyprus,Canada,Malawi,Saudi Arabia,Bosnia and Herzegovina,Ethiopia,Spain,Slovenia,Oman,Saint Pierre and Miquelon,Macau,San Marino,Lesotho,Marshall Islands,Sint Maarten,Iceland,Luxembourg,Argentina,Turks and Caicos Islands,Nauru,Cocos (Keeling) Islands,Western Sahara,Dominica,Costa Rica,Australia,Thailand,Haiti,Tuvalu,Honduras,Equatorial Guinea,Saint Lucia,French Polynesia,Belarus,Latvia,Palau,Guadeloupe,Philippines,Gibraltar,Denmark,Cameroon,Guinea,Bahrain,Suriname,DR Congo,Somalia,Czechia,New Caledonia,Vanuatu,"Saint Helena, Ascension and Tristan da Cunha",Togo,British Virgin Islands,Kenya,Niue,Heard Island and McDonald Islands,Rwanda,Estonia,Romania,Trinidad and Tobago,Guyana,Timor-Leste,Vietnam,Uruguay,Vatican City,Hong Kong,Austria,Antigua and Barbuda,Turkmenistan,Mozambique,Panama,Micronesia,Ireland,Curaçao,French Guiana,Norway,Åland Islands,Central African Republic,Burkina Faso,Eritrea,Tanzania,South Korea,Jordan,Mauritania,Lithuania,United States Minor Outlying Islands,Slovakia,Angola,Kazakhstan,Moldova,Mali,Falkland Islands,Armenia,Samoa,Jersey,Japan,Bolivia,Chile,United States,Saint Vincent and the Grenadines,Bermuda,Seychelles,British Indian Ocean Territory,Guatemala,Ecuador,Martinique,Tajikistan,Malta,Gambia,Nigeria,Bahamas,Kosovo,Kuwait,Maldives,South Sudan,Iran,Albania,Brazil,Serbia,Belize,Myanmar,Bhutan,Venezuela,Liberia,Jamaica,Poland,Cayman Islands,Brunei,Comoros,Guam,Tonga,Kiribati,Ghana,Chad,Zimbabwe,Saint Martin,Mongolia,Portugal,American Samoa,Republic of the Congo,Belgium,Israel,New Zealand,Nicaragua,Anguilla
continent,Antarctica,North America,Europe,Africa,Europe,Asia,Oceania,North America,Oceania,Africa,Africa,Europe,Africa,Asia,Africa,North America,Asia,North America,Africa,Europe,Africa,Africa,Europe,Africa,North America,Africa,Oceania,Europe,Asia,Asia,North America,Europe,Oceania,Europe,Europe,Asia,Africa,Africa,Asia,Asia,Asia,South America,Asia,Europe,Asia,North America,Oceania,Europe,Africa,Europe,Africa,Asia,Oceania,Africa,Europe,Africa,Africa,Asia,Africa,North America,North America,South America,Europe,Europe,Africa,Af